<a href="https://colab.research.google.com/github/jorge774/Procesamiento_datos/blob/main/procesar_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''procesar_datos.py
   Este programa hace lo siguiente:
   1.Reavisa la carpeta donde estan los archivos txt que genera janus luego de hacer adquisicion.
   2.Revisa y toma los archivos txt que no ha leido anteriormente.
   3.Toma el encabezado del archivo RunXX_list.txt con menor XX de todo el lote de archivos tomados en 2.
   4.Toma como referencia temporal el 'Run start time' del encabezado tomado en 3.
   5.Genera una estampa temporal para cada evento de todos los archivos RunXX_list.txt tomando como referencia a 4.
   6.Genera un archivo 'MuTe_20240304_11h07m01s.txt'
   7.Elimina aquellos eventos donde el conteo en todos los canales fue 0.
   8.Elimina los archivos RunXX_list.txt

   Los parametros de este programa son leidos en el archivo 'config.txt', los cuales son:
   1.Ruta absoluta donde quedan los archivos txt que genera janus luego de hacer adquisicion.
   2.Ruta absoluta donde quedaran los archivos 'MuTe_20240304_11h07m01s.txt'.
   3.Numero de canales usados en la adquision.
   4.Numero de lineas usadas en los encabezados de los archivos 'RunXX_list.txt'.

   En el archivo 'vars.txt' el programa escribe el nombre de los archivos 'RunXX_list.txt' que va procesando.
'''

In [ ]:
print('procesando...')
import os
import re
import time
from datetime import datetime
from datetime import timedelta
from datetime import timezone
#from pytz import timezone
ini_exe=datetime.now(timezone.utc)
def key_function(string):return int(re.search(pattern='[0-9]{1,}',string=string).group())

In [ ]:
config_file=open(file='config.txt',mode='r')
output_input_folders=config_file.read()
config_file.close()

folder_path_caen=output_input_folders.split('\n')[0].split('#')[0]
folder_path_vars='vars.txt'
output_folder_path=output_input_folders.split('\n')[1].split('#')[0]


##ini main loop
vars_file=open(file=folder_path_vars,mode='r')
processed_file_list=vars_file.read()
vars_file.close()
current_file_list=os.listdir(folder_path_caen)

if processed_file_list:
   #processed_file_list is not empty
   files_to_proces=list(set(current_file_list)-set(processed_file_list.split('\n')))#archivos que esten en folder_path_caen pero que no esten en el vars file
else:
   #processed_file_list is empty
   files_to_proces=current_file_list

files_to_proces.sort(key=key_function)#ordenar los files_to_proces de acuerdo al numero de corrida
if files_to_proces:
    #files_to_proces is not empty
    #n_files_to_process=int(output_input_folders.split('\n')[4].split('#')[0])
    count_files_run_list_processed=0
    last_file_to_proces=False
    total_num_of_Runxx_list_txt_files=sum(1 for i in files_to_proces if re.search('list.txt',i))
    #'MuTe_20240304_11h07m01s'
    #utc_datetime=datetime.utcnow()
    utc_datetime=datetime.now(timezone.utc)
    cur_h='0'+str(utc_datetime.hour) if utc_datetime.hour<10 else str(utc_datetime.hour)
    cur_day='0'+str(utc_datetime.day) if utc_datetime.day<10 else str(utc_datetime.day)
    cur_mon='0'+str(utc_datetime.month) if utc_datetime.month<10 else str(utc_datetime.month)
    cur_min='0'+str(utc_datetime.minute) if utc_datetime.minute<10 else str(utc_datetime.minute)
    cur_sec='0'+str(utc_datetime.second) if utc_datetime.second<10 else str(utc_datetime.second)
    name='MuTe_'+str(utc_datetime.year)+cur_mon+cur_day+'_'+cur_h+'h'+cur_min+'m'+cur_sec+'s'+'.txt'
    for file in files_to_proces:
        if re.search('list.txt',file):
           if count_files_run_list_processed==(total_num_of_Runxx_list_txt_files-1):last_file_to_proces=True
           #procesar los datos adquiridos
           #se procesa un archivo Runxx_list.txt de files_to_proces
           #print(file)
           #ejecutar con cada archivo Runxx_list.txt
           num_header_lines=int(output_input_folders.split('\n')[3].split('#')[0])#8
           n_channels=int(output_input_folders.split('\n')[2].split('#')[0])#64 #numero de canales
           file_path=folder_path_caen+'/'+file
           file_to_proces=open(file=file_path,mode='r')
           data_and_header=file_to_proces.readlines()
           file_to_proces.close()
           output_file_path=output_folder_path+'/'+name
           output_file=open(file=output_file_path,mode='a')

           header=data_and_header[:num_header_lines]
           if count_files_run_list_processed==0:
              output_file.writelines(header)
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              ref_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')
              delta_time=ref_date_time-ref_date_time
              #delta_time=str(delta_time).split(sep=':')
              #delta_time=[int(i) for i in delta_time]
              #delta_time.append(0)

              aux=['ch0'+str(i) if i<10 else 'ch'+str(i) for i in range(n_channels)]
              aux2=[aux[i]+' ' if i!=(len(aux)-1) else aux[i] for i in range(len(aux))];#header2='Tstamp_us           '#11 espacios en blanco
              header2='h  '+'m  '+'s  '+'us         '#9 espacios en blancos+2espacios por 'us'=10 espacios
              for i in aux2:header2+=i
              output_file.write(header2+'\n')
           else:
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              last_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')
              delta_time=last_date_time-ref_date_time
              #delta_time=str(delta_time).split(sep=':')
              #delta_time=[int(i) for i in delta_time]
              #delta_time.append(0)

           ini_index=num_header_lines+1
           end_index=num_header_lines+n_channels+1
           while ini_index<(len(data_and_header)-1):
                #recorrer todos los data blocks de un archivo Runxx_list.txt
                data_block=data_and_header[ini_index:end_index]
                #ejeutar para cada data block
                Tstamp_us=data_block[0].split()[0]

                mircrosegundos=float(Tstamp_us)+(delta_time.total_seconds()*1e+6)
                #horas,micorosegundos sobrantes
                horas=mircrosegundos//3.6e+9;mircrosegundos=mircrosegundos%3.6e+9

                #minutos, micorosegundos sobrantes
                minutos=mircrosegundos//6e+7;mircrosegundos=mircrosegundos%6e+7

                #segundos, micorosegundos sobrantes
                segundos=mircrosegundos//1e+6;mircrosegundos=mircrosegundos%1e+6

                #horas,minutos,segundos,microsegundos
                time_stamp_to_print=[int(horas),int(minutos),int(segundos),round(mircrosegundos,3)]

                h='0'+str(time_stamp_to_print[0]) if (time_stamp_to_print[0]<10) else str(time_stamp_to_print[0])
                m='0'+str(time_stamp_to_print[1]) if (time_stamp_to_print[1]<10) else str(time_stamp_to_print[1])
                s='0'+str(time_stamp_to_print[2]) if (time_stamp_to_print[2]<10) else str(time_stamp_to_print[2])
                us=str(time_stamp_to_print[3])

                output_file.write(h);output_file.write(' ')
                output_file.write(m);output_file.write(' ')
                output_file.write(s);output_file.write(' ')
                output_file.write(us);output_file.write(' ')
                for _ in range(10+1-len(list(us))):output_file.write(' ')#10 espacios-numero de caracteres en us+1 espacio
                for i in range(len(data_block)):
                    count=data_block[i].split()[-1]
                    if i!=(len(data_block)-1):
                      output_file.write(count)
                      for _ in range(4+1-len(list(count))):output_file.write(' ')
                    else:
                      output_file.write(count+'\n')
                ini_index=end_index
                end_index=ini_index+n_channels
           output_file.close()
           count_files_run_list_processed+=1
           os.remove(file_path)#eliinar archivo runxxlits.txt
        if re.search('Info.txt',file):
           #procesar metadata de los datos adquiridos
           pass
        #if count_files_run_list_processed==n_files_to_process: break
    del data_and_header
    vars_file=open(file=folder_path_vars,mode='a')
    if processed_file_list:
       #processed_file_list is not empty
       vars_file.write('\n')
       for i in range(0,len(files_to_proces)):
           if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
           else:
              vars_file.write(files_to_proces[i])
    else:
      #processed_file_list is empty
      for i in range(0,len(files_to_proces)):
          if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
          else:
              vars_file.write(files_to_proces[i])
    vars_file.close()
    output_file=open(file=output_file_path,mode='r')
    content_output_file=output_file.readlines()
    output_file.close()
    for nth_line in range(num_header_lines+1,len(content_output_file)):
        if content_output_file[nth_line].split()[4:].count('0')==n_channels:
           content_output_file[nth_line]=''#eliminar linea que tiene todos los ch en 0
    if content_output_file[num_header_lines+1:].count('')==len(content_output_file)-(num_header_lines+1):
       os.remove(output_file_path)#eliminar txt si esta vacio
    else:
       output_file=open(file=output_file_path,mode='w')
       output_file.writelines(content_output_file)
       output_file.close()
    del content_output_file
print('Duracion del procesamiento:',(datetime.now(timezone.utc)-ini_exe).total_seconds(),'segundos')
##end main loop

Run2_list.txt
Run3_list.txt
Run10_list.txt
